### Install keras and tensorflow:

In [0]:
!conda install -c conda-forge keras --yes
!conda install -c conda-forge tensorflow --yes

### Import libraries:

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import os

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Data import:

In [0]:
folder_path = 'food/'

img_width, img_height = 224, 224

# load all images into a list
file_list = sorted(os.listdir(folder_path))
images = []
for img in file_list:
    if img == '.DS_Store':  # ignore stupid fookin mac file that wont go away
        continue
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)

### Load pre-trained model:

In [0]:
model = ResNet50(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


### Predict:

In [0]:
predictions = model.predict(images, batch_size=64, verbose=1)

10000/10000 [==============================] - 3732s 373ms/sample


### Save:

In [0]:
# Save top 3 label probabilities as the 3 features describing image:
predictions_decoded = np.array(decode_predictions(predictions, top=3))
predictions_decoded = predictions_decoded[:, :, 1]

df = pd.DataFrame(predictions_decoded, columns=['first', 'second', 'third'])
print(df)

df.to_csv('food_features', index=False)

           first         second         third
0        thimble      trilobite        packet
1         potpie         bakery         dough
2      ice_cream  mashed_potato         dough
3      ice_cream          dough     meat_loaf
4        burrito        pretzel     carbonara
...          ...            ...           ...
9995   ice_cream     neck_brace      lipstick
9996  strawberry      ice_cream      baseball
9997   meat_loaf          pizza  wooden_spoon
9998   soup_bowl     frying_pan         loupe
9999       dough           wool        mortar

[10000 rows x 3 columns]


In [0]:
'''img_path = 'food/00000.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x)
pred = np.array(decode_predictions(pred, top=3))
print(pred)'''

[[['n04423845' 'thimble' '0.7259098']
  ['n01768244' 'trilobite' '0.102758445']
  ['n03871628' 'packet' '0.019814285']]]


# Load food features and one hot encode:

Now we need to load the newly generated features, train a neural net on the triplets with the new features and see what happens:)

Load food features:

In [2]:
df_features = pd.read_csv("food_features") 
print(df_features)

           first         second         third
0        thimble      trilobite        packet
1         potpie         bakery         dough
2      ice_cream  mashed_potato         dough
3      ice_cream          dough     meat_loaf
4        burrito        pretzel     carbonara
...          ...            ...           ...
9995   ice_cream     neck_brace      lipstick
9996  strawberry      ice_cream      baseball
9997   meat_loaf          pizza  wooden_spoon
9998   soup_bowl     frying_pan         loupe
9999       dough           wool        mortar

[10000 rows x 3 columns]


Load triplets:

In [53]:
df_triplets = pd.read_csv('train_triplets.txt', sep=" ", header=None)
print(df_triplets)

          0     1     2
0      2461  3450  2678
1      2299  2499  4987
2      4663  1056  3029
3      4532  1186  1297
4      3454  3809  2204
...     ...   ...   ...
59510   466  2952  2530
59511  2646  3580  2359
59512  3255  4844  4334
59513  2136  4619   161
59514  2509  2552  3406

[59515 rows x 3 columns]


Somewhere around here we need to split into training and validation set. 

The issue is that the validation set triplets and training set triplets should not include any of the same images for the evaluation to be accurate. 

Maybe the easiest way to do this is to take some portion of the data as train set, and then iterate through the rest and if none of the three images are in the train set it goes in the validation set, otherwise if none of the three images are in the validation set it goes in the train set, otherwise it is discarded? Then tune the size of the initial train set to get ish the percent of the split we want? Very sub-optimal...

In [54]:
df_triplets_train = pd.DataFrame(columns = [0, 1, 2])
df_triplets_val = pd.DataFrame(columns = [0, 1, 2])

N = 325
for idx, row in df_triplets.iterrows():
  if idx < N:  # first N rows go in train
    df_triplets_train = df_triplets_train.append(row)
  elif all(x not in df_triplets_train.values for x in [row[0], row[1], row[2]]):  # else if images not in train add to val
    df_triplets_val = df_triplets_val.append(row)
  elif all(x not in df_triplets_val.values for x in [row[0], row[1], row[2]]):  # else if images not in val add to train
    df_triplets_train = df_triplets_train.append(row)
  # else we discard triplet

print(df_triplets_train)
print(df_triplets_val)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
          0     1     2
0      2461  3450  2678
1      2299  2499  4987
2      4663  1056  3029
3      4532  1186  1297
4      3454  3809  2204
...     ...   ...   ...
59500   450   219  4712
59503   236  1229  4090
59509   880  1818   621
59510   466  2952  2530
59511  2646  3580  2359

[15892 rows x 3 columns]
          0     1     2
328     647  4571  3011
330     683  2848  2039
331    4404  2384  2478
334    1019  4241  4114
338    4108  1020  1523
...     ...   ...   ...
59465  4020  2390  1285
59468  4990  4028  2647
59475  1716  4431  3398
59481  1550  1222  1140
59496  1940    38  2869

[5005 rows x 3 columns]


Problem: all the training data has label 1 right now.

Attempt at solution: swap B and C for every other row, such that the labels are [1 0 1 0 1 ...].

In [26]:
for i, row in df_triplets_train.iterrows():
  if i % 2 == 1:
    temp = row[1]
    df_triplets_train.at[i,1] = row[2]
    df_triplets_train.at[i,2] = temp
print(df_triplets_train)

          0     1     2
0      2461  3450  2678
1      2299  2499  4987
2      4663  1056  3029
3      4532  1186  1297
4      3454  3809  2204
...     ...   ...   ...
15887   450  4712   219
15888   236  4090  1229
15889   880  1818   621
15890   466  2952  2530
15891  2646  3580  2359

[15892 rows x 3 columns]


In [58]:
df_triplets = pd.concat([df_triplets_train, df_triplets_val])  # Put the two dataframes back on top of each other

df_triplets = df_triplets.reset_index(drop=True)  # Reset index column

print(df_triplets)

          0     1     2
0      2461  3450  2678
1      2299  2499  4987
2      4663  1056  3029
3      4532  1186  1297
4      3454  3809  2204
...     ...   ...   ...
20892  4020  2390  1285
20893  4990  4028  2647
20894  1716  4431  3398
20895  1550  1222  1140
20896  1940    38  2869

[20897 rows x 3 columns]


Generate dataset:

In [59]:
def generate_dataset(df_triplets, df_features):
  a = np.empty(shape=(df_triplets.shape[0], 9), dtype='object')

  for i in range(3):
    for idx, x in df_triplets[i].iteritems():
      a[idx, 3*i] = df_features.iloc[x, 0]
      a[idx, 3*i+1] = df_features.iloc[x, 1]
      a[idx, 3*i+2] = df_features.iloc[x, 2]

  df_triplets['A_1'], df_triplets['A_2'], df_triplets['A_3'] = [a[:, 0], a[:, 1], a[:, 2]]
  df_triplets['B_1'], df_triplets['B_2'], df_triplets['B_3'] = [a[:, 3], a[:, 4], a[:, 5]]
  df_triplets['C_1'], df_triplets['C_2'], df_triplets['C_3'] = [a[:, 6], a[:, 7], a[:, 8]]

  #df_triplets = df_triplets.drop(columns=[0, 1, 2])

  return df_triplets

df_triplets = generate_dataset(df_triplets, df_features)
print(df_triplets)

          0     1     2  ...               C_1               C_2           C_3
0      2461  3450  2678  ...              tray        strawberry      cucumber
1      2299  2499  4987  ...        frying_pan               wok  wooden_spoon
2      4663  1056  3029  ...       cauliflower        rotisserie         plate
3      4532  1186  1297  ...  hen-of-the-woods      coral_fungus     gyromitra
4      3454  3809  2204  ...             plate  butternut_squash  acorn_squash
...     ...   ...   ...  ...               ...               ...           ...
20892  4020  2390  1285  ...       French_loaf            bakery         dough
20893  4990  4028  2647  ...         meat_loaf     mashed_potato   mixing_bowl
20894  1716  4431  3398  ...            bolete             plate        potpie
20895  1550  1222  1140  ...          bolo_tie       waffle_iron     meat_loaf
20896  1940    38  2869  ...         guacamole           burrito         plate

[20897 rows x 12 columns]


*One*-hot encode:

In [62]:
df_triplets = df_triplets.drop(columns=[0, 1, 2])  # remove original image columns
print(df_triplets)

                 A_1               A_2  ...               C_2           C_3
0          ice_lolly        strawberry  ...        strawberry      cucumber
1      mashed_potato         ice_cream  ...               wok  wooden_spoon
2             orange               hip  ...        rotisserie         plate
3            pretzel             bagel  ...      coral_fungus     gyromitra
4              acorn               hip  ...  butternut_squash  acorn_squash
...              ...               ...  ...               ...           ...
20892          dough            bakery  ...            bakery         dough
20893     neck_brace            potpie  ...     mashed_potato   mixing_bowl
20894      meat_loaf       French_loaf  ...             plate        potpie
20895           corn               ear  ...       waffle_iron     meat_loaf
20896            ear  spaghetti_squash  ...           burrito         plate

[20897 rows x 9 columns]


In [63]:
df_triplets_oh = pd.get_dummies(data=df_triplets)
print(df_triplets_oh)

       A_1_African_chameleon  A_1_African_crocodile  ...  C_3_wreck  C_3_zucchini
0                          0                      0  ...          0             0
1                          0                      0  ...          0             0
2                          0                      0  ...          0             0
3                          0                      0  ...          0             0
4                          0                      0  ...          0             0
...                      ...                    ...  ...        ...           ...
20892                      0                      0  ...          0             0
20893                      0                      0  ...          0             0
20894                      0                      0  ...          0             0
20895                      0                      0  ...          0             0
20896                      0                      0  ...          0             0

[20897 rows x 2

In [64]:
df_triplets_train_oh = df_triplets_oh.iloc[:15892, :]
df_triplets_val_oh = df_triplets_oh.iloc[15892:, :]

print(df_triplets_train)
print(df_triplets_val)

          0     1     2
0      2461  3450  2678
1      2299  2499  4987
2      4663  1056  3029
3      4532  1186  1297
4      3454  3809  2204
...     ...   ...   ...
15887   450   219  4712
15888   236  1229  4090
15889   880  1818   621
15890   466  2952  2530
15891  2646  3580  2359

[15892 rows x 3 columns]
         0     1     2
0      647  4571  3011
1      683  2848  2039
2     4404  2384  2478
3     1019  4241  4114
4     4108  1020  1523
...    ...   ...   ...
5000  4020  2390  1285
5001  4990  4028  2647
5002  1716  4431  3398
5003  1550  1222  1140
5004  1940    38  2869

[5005 rows x 3 columns]


In [36]:
# Save completed train and validation data sets:
df_triplets_train.to_csv('train', index=False)
df_triplets_val.to_csv('val', index=False)

                 A_1            A_2  ...               C_2            C_3
0          ice_lolly     strawberry  ...        strawberry       cucumber
1      mashed_potato      ice_cream  ...     jigsaw_puzzle     coral_reef
2             orange            hip  ...        rotisserie          plate
3            pretzel          bagel  ...         meat_loaf     frying_pan
4              acorn            hip  ...  butternut_squash   acorn_squash
...              ...            ...  ...               ...            ...
14564          plate      meat_loaf  ...        frying_pan          bagel
14565          plate  mashed_potato  ...   chocolate_sauce      ice_cream
14566        burrito         potpie  ...             plate           corn
14567           corn         bucket  ...             dough  custard_apple
14568          plate  mashed_potato  ...              maze    French_loaf

[14569 rows x 9 columns]
               A_1               A_2  ...            C_2               C_3
0        se

# Train net:

Load:

In [0]:
x_train = pd.read_csv('train_triplets_oh')
print(x_train)

       A_1_African_chameleon  A_1_African_crocodile  ...  C_3_wreck  C_3_zucchini
0                          0                      0  ...          0             0
1                          0                      0  ...          0             0
2                          0                      0  ...          0             0
3                          0                      0  ...          0             0
4                          0                      0  ...          0             0
...                      ...                    ...  ...        ...           ...
59510                      0                      0  ...          0             0
59511                      0                      0  ...          0             0
59512                      0                      0  ...          0             0
59513                      0                      0  ...          0             0
59514                      0                      0  ...          0             0

[59515 rows x 3

Generate labels:

In [0]:
y_train = np.empty((x_train.shape[0],1))
y_train[::2] = 1
y_train[1::2] = 0
print(y_train)

[[1.]
 [0.]
 [1.]
 ...
 [1.]
 [0.]
 [1.]]


Train:

In [0]:
model = Sequential()
model.add(layers.Dense(800, input_dim=x_train.shape[1], activation='relu'))
#model.add(Dropout(0.2))
model.add(layers.Dense(800, activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train.values, y_train, epochs=5, batch_size=32, validation_split=0.2, shuffle=True)

Epoch 1/5
1488/1488 [==============================] - 38s 25ms/step - loss: 0.6474 - accuracy: 0.6086 - val_loss: 0.5970 - val_accuracy: 0.6651
Epoch 2/5
1488/1488 [==============================] - 37s 25ms/step - loss: 0.4481 - accuracy: 0.7876 - val_loss: 0.5363 - val_accuracy: 0.7459
Epoch 3/5
1488/1488 [==============================] - 38s 25ms/step - loss: 0.1246 - accuracy: 0.9519 - val_loss: 0.6761 - val_accuracy: 0.7770
Epoch 4/5
1488/1488 [==============================] - 38s 25ms/step - loss: 0.0239 - accuracy: 0.9919 - val_loss: 0.9484 - val_accuracy: 0.7846
Epoch 5/5
1488/1488 [==============================] - 38s 25ms/step - loss: 0.0101 - accuracy: 0.9966 - val_loss: 1.1446 - val_accuracy: 0.7894


Hard: 0.688633839319

Medium: 	0.627444380869

Easy: 	0.578405304433

Generate test data:


In [0]:
# generate test data here

Predict:

In [0]:
predictions = model.predict(x_test)